In [ ]:
%pip install imageio opencv-python alphashape

In [ ]:
# import libraries
import scanpy as sc
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
from shapely.ops import transform
from shapely.affinity import scale
import tifffile as tiff
import shapely.affinity as sa
import cv2
import json
import imageio as io
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
from matplotlib.colors import Normalize
from matplotlib.cm import coolwarm

In [ ]:
import alphashape


def make_alphashape(points: pd.DataFrame, alpha: float):
    """
    Create a cell boundary with alpha shape from the provided points

    Args:
        points (pd.DataFrame): dataframe with columns "x" and "y" for the positions of the transcripts
        alpha (float): alpha value for the alpha shape

    Returns:
        shape (shapely.geometry.Polygon): alpha shape cell segmentation boundary
    """
    points = np.array(points)
    shape = alphashape.alphashape(points, alpha=alpha)
    return shape

### Things that we may want to overlay:
##### Transcripts, Aligned IF, Xenium DAPI, H and E, cell segmentation, transcripts/cell masks colored by cell type

In [ ]:
experiment_name = "human_09_r2"

### Loading in adata
path_to_main_adata = "../data/adata/human.h5ad"

whole_adata = sc.read(path_to_main_adata)
finalized_adata = whole_adata[whole_adata.obs.batch == experiment_name]

# the following has the transcripts saved. It is a temporary adata along the processing pipeline
path_to_adata_with_transcripts = "../data/adata/human_09_r2_with_transcripts.h5ad"

# all h and e and IF are generated and saved
path_to_h_and_e = "../data/images/human_09_r2_h_and_e_alignment_gan.npy"
path_to_if = "../data/images/human_09_r2_IF_alignment.npy"

xenium_output_path = "../data/xenium_output/human_09_r2"

In [ ]:
# define the colors
finalized_adata.uns["Subtype_colors"] = [
    "#ffff00",
    "#1ce6ff",
    "#ff34ff",
    "#ff4a46",
    "#008941",
    "#006fa6",
    "#a30059",
    "#ffdbe5",
    "#7a4900",
    "#0000a6",
    "#63ffac",
    "#b79762",
    "#004d43",
    "#8fb0ff",
    "#997d87",
    "#5a0007",
    "#809693",
    "#6a3a4c",
    "#1b4400",
    "#4fc601",
    "#3b5dff",
    "#4a3b53",
    "#ff2f80",
    "#61615a",
    "#ba0900",
    "#6b7900",
    "#00c2a0",
    "#ffaa92",
    "#ff90c9",
    "#b903aa",
    "#d16100",
    "#ddefff",
    "#000035",
    "#7b4f4b",
    "#a1c299",
    "#300018",
    "#0aa6d8",
]

In [ ]:
def import_image(path: str):
    file = os.path.join(path, "morphology_mip.ome.tif")
    img = io.imread(file)
    return img


xenium_dapi = import_image(xenium_output_path)

try:
    IF_image = np.load(path_to_if)
except:
    print("No IF for this experiment")
    IF_image = xenium_dapi

try:
    h_an_e = np.load(path_to_h_and_e)
except:
    print("No H&E for this experiment")
    h_an_e = xenium_dapi

In [ ]:
# Read in the adata holding the transcripts
transcripts = sc.read(path_to_adata_with_transcripts)
points = transcripts.uns["points"]

In [ ]:
##save the different parts of the transcripts df for fast indexing
points_x = points.x.values
points_y = points.y.values
points_z = points.z.values
points_gene = points.gene.values
points_cell = points.cell.values
points_split_cell = points.split_cell.values
points["split_cell"] = points["split_cell"].values.astype(int)

In [ ]:
def get_pixel_size(path: str) -> float:
    file = open(os.path.join(path, "experiment.xenium"))
    experiment = json.load(file)
    pixel_size = experiment["pixel_size"]
    return pixel_size


# transform the transcript coordinates from microns to pixels
pixel_size = get_pixel_size(xenium_output_path)
transformed_x = points_x * (1 / pixel_size)
transformed_y = points_y * (1 / pixel_size)

In [ ]:
# Showing the xenium dapi overview
plt.imshow(xenium_dapi)
plt.xlabel("min_y ----------------------- max_y")
plt.ylabel("max_x ----------------------- min_x")
plt.grid(True)
plt.show()

In [ ]:
# Downscaling the dapi overview by 50x (you can change this with no side effects other than runtime in the next cell)

down_factor = 50

new_width = int(xenium_dapi.shape[1] / down_factor)
new_height = int(xenium_dapi.shape[0] / down_factor)

thumbnail = cv2.resize(xenium_dapi, (new_width, new_height))

In [ ]:
min_y = 667
max_y = 688

min_x = 415
max_x = 445

In [ ]:
# Get the transcripts falling in the box you created

min_x = min_x * down_factor
min_y = min_y * down_factor
max_x = max_x * down_factor
max_y = max_y * down_factor


subsetted_indices = np.where(
    (transformed_x > min_y)
    & (transformed_x < max_y)
    & (transformed_y > min_x)
    & (transformed_y < max_x)
)[0]

transcripts_df = pd.DataFrame(
    zip(
        transformed_x[subsetted_indices],
        transformed_y[subsetted_indices],
        points_gene[subsetted_indices],
        points_split_cell[subsetted_indices],
    ),
    index=points_cell[subsetted_indices],
    columns=["x", "y", "gene", "split_cell"],
)

In [ ]:
plot_down = 4

## Part 1: Money shot

In [ ]:
section_min_y = 580
section_max_y = 760

section_min_x = 380
section_max_x = 560

In [ ]:
rectangle_min_y = 667
rectangle_max_y = 690

rectangle_min_x = 415
rectangle_max_x = 445

In [ ]:
# Get the transcripts falling in the box you created

rectangle_min_x = rectangle_min_x * down_factor
rectangle_min_y = rectangle_min_y * down_factor
rectangle_max_x = rectangle_max_x * down_factor
rectangle_max_y = rectangle_max_y * down_factor

In [ ]:
# Get the transcripts falling in the box you created

section_min_x = section_min_x * down_factor
section_min_y = section_min_y * down_factor
section_max_x = section_max_x * down_factor
section_max_y = section_max_y * down_factor


section_subsetted_indices = np.where(
    (transformed_x > section_min_y)
    & (transformed_x < section_max_y)
    & (transformed_y > section_min_x)
    & (transformed_y < section_max_x)
)[0]

section_transcripts_df = pd.DataFrame(
    zip(
        transformed_x[section_subsetted_indices],
        transformed_y[section_subsetted_indices],
        points_gene[section_subsetted_indices],
        points_split_cell[section_subsetted_indices],
    ),
    index=points_cell[section_subsetted_indices],
    columns=["x", "y", "gene", "split_cell"],
)

In [ ]:
segmentation_face_color = "Subtype"
inside_alpha = 0.34
outside_alpha = 0.34
celltypes = []
ids = np.array([i.split("-")[0] for i in finalized_adata.obs.index.values]).astype(int)
id_df = pd.DataFrame(
    zip(ids, finalized_adata.obs[segmentation_face_color].values),
    columns=["id", segmentation_face_color],
)
section_transcripts_with_obs = section_transcripts_df.merge(
    id_df, left_on="split_cell", right_on="id", how="left"
)
section_transcripts_with_obs = section_transcripts_with_obs.dropna(axis=0)


print("Making Shapes")
gby = section_transcripts_with_obs[
    (section_transcripts_with_obs.split_cell != 0)
    & (section_transcripts_with_obs.split_cell != -1)
].groupby("split_cell")


shapes = []
for group in tqdm(gby):
    shapes.append(make_alphashape(group[1][["x", "y"]].values, alpha=0.05))
    ctype = group[1][segmentation_face_color].values[0]
    cell_location = np.where(
        finalized_adata.obs[segmentation_face_color].cat.categories == ctype
    )[0]
    try:
        celltypes.append(
            finalized_adata.uns[f"{segmentation_face_color}_colors"][cell_location][0]
        )
    except:
        celltypes.append(
            finalized_adata.uns[f"{segmentation_face_color}_colors"][cell_location[0]]
        )
shapes = gpd.GeoSeries(shapes)
colors = celltypes

# Create an empty GeoDataFrame to store adjusted polygons
adjusted_shapes = []

# Iterate through the shapes DataFrame and adjust each polygon
for original_polygon in shapes:
    scaled_polygon = sa.translate(original_polygon, -section_min_y, -section_min_x)
    adjusted_shapes.append(scaled_polygon)

adjusted_shapes = gpd.GeoSeries(adjusted_shapes)

In [ ]:
plt.figure(figsize=(10, 4), dpi=300)
ax1 = plt.gca()

img_cropped = xenium_dapi[
    section_min_x:section_max_x, section_min_y:section_max_y
]  # [second_min_x:second_max_x, second_min_y:second_max_y]
ax1.imshow(img_cropped, vmax=np.percentile(img_cropped, 99.9), cmap="Greys_r")

for geometry, color in zip(adjusted_shapes, colors):
    if geometry.geom_type == "Polygon":
        patch = plt.Polygon(
            list(zip(*geometry.exterior.xy)),
            facecolor=color,
            edgecolor="none",
            alpha=inside_alpha,
            zorder=1,
        )
        ax1.add_patch(patch)
    elif geometry.geom_type == "MultiPolygon":
        for poly in geometry:
            patch = plt.Polygon(
                list(zip(*poly.exterior.xy)),
                facecolor=color,
                edgecolor="none",
                alpha=inside_alpha,
                zorder=1,
            )
            ax1.add_patch(patch)

# Plot polygon edges with edgecolor based on data values
for geometry, color in zip(adjusted_shapes, colors):
    if geometry.geom_type == "Polygon":
        ax1.plot(*geometry.exterior.xy, color=color, alpha=outside_alpha, linewidth=0.2)
    elif geometry.geom_type == "MultiPolygon":
        for poly in geometry:
            ax1.plot(*poly.exterior.xy, color=color, alpha=outside_alpha, linewidth=0.2)

rectangle2 = Rectangle(
    (rectangle_min_y - section_min_y, rectangle_min_x - section_min_x),
    rectangle_max_y - rectangle_min_y,
    rectangle_max_x - rectangle_min_x,
    linewidth=3,
    edgecolor="white",
    facecolor="none",
    zorder=2,
)
ax1.add_patch(rectangle2)
ax1.set_xlim(0, section_max_y - section_min_y)
ax1.set_ylim(0, section_max_x - section_min_x)
ax1.invert_yaxis()
# ax1.axis('equal')
ax1.axis("off")
plt.show()

## Part 2: HE image

In [ ]:
second_min_y = 810
second_max_y = 885

second_min_x = 645
second_max_x = 780

side1 = min_y + second_min_y
side2 = max_y - (max_y - (min_y + second_max_y))
side3 = second_min_x + min_x
side4 = max_x - (max_x - (second_max_x + min_x))

subsetted_indices_second = np.where(
    (transformed_x > side1)
    & (transformed_x < side2)
    & (transformed_y > side3)
    & (transformed_y < side4)
)[0]

transcripts_df_second = pd.DataFrame(
    zip(
        transformed_x[subsetted_indices_second],
        transformed_y[subsetted_indices_second],
        points_gene[subsetted_indices_second],
        points_split_cell[subsetted_indices_second],
    ),
    index=points_cell[subsetted_indices_second],
    columns=["x", "y", "gene", "split_cell"],
)

In [ ]:
import cv2
import numpy as np


def adjust_white_pixels(image, threshold=248):
    # Convert image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the range of white color in HSV
    lower_white = np.array([0, 0, threshold])
    upper_white = np.array([255, 5, 255])

    # Threshold the HSV image to get only white colors
    mask = cv2.inRange(hsv_image, lower_white, upper_white)

    # Replace white pixels in the original image with pure white
    image[mask == 255] = [255, 255, 255]

    return image

In [ ]:
plt.figure(figsize=(10, 4), dpi=300)
ax3 = plt.gca()
# Assuming 'thumbnail' is your image data
img_cropped = h_an_e[min_x:max_x, min_y:max_y, :]


# Call the function to adjust white pixels
output_image = adjust_white_pixels(img_cropped.copy())


ax3.imshow(output_image)

# Add a black rectangle
rectangle = Rectangle(
    (second_min_y, second_min_x),
    second_max_y - second_min_y,
    second_max_x - second_min_x,
    linewidth=3,
    edgecolor="black",
    facecolor="none",
)
ax3.add_patch(rectangle)
ax3.axis("off")

plt.show()

## Part 3: Leiden

In [ ]:
import alphashape


def make_alphashape(points: pd.DataFrame, alpha: float):
    points = np.array(points)
    shape = alphashape.alphashape(points, alpha=alpha)
    return shape


plt.figure(figsize=(10, 4), dpi=300)
ax1 = plt.gca()

segmentation_face_color = "leiden"
inside_alpha = 0.34
outside_alpha = 0.34
celltypes = []
ids = np.array([i.split("-")[0] for i in finalized_adata.obs.index.values]).astype(int)
id_df = pd.DataFrame(
    zip(ids, finalized_adata.obs[segmentation_face_color].values),
    columns=["id", segmentation_face_color],
)
transcripts_with_obs = transcripts_df.merge(
    id_df, left_on="split_cell", right_on="id", how="left"
)
transcripts_with_obs = transcripts_with_obs.dropna(axis=0)


print("Making Shapes")
gby = transcripts_with_obs[
    (transcripts_with_obs.split_cell != 0) & (transcripts_with_obs.split_cell != -1)
].groupby("split_cell")


shapes = []
for group in tqdm(gby):
    shapes.append(make_alphashape(group[1][["x", "y"]].values, alpha=0.05))
    ctype = group[1][segmentation_face_color].values[0]
    cell_location = np.where(
        finalized_adata.obs[segmentation_face_color].cat.categories == ctype
    )[0]
    try:
        celltypes.append(
            finalized_adata.uns[f"{segmentation_face_color}_colors"][cell_location][0]
        )
    except:
        celltypes.append(
            finalized_adata.uns[f"{segmentation_face_color}_colors"][cell_location[0]]
        )
shapes = gpd.GeoSeries(shapes)
colors = celltypes


img_cropped = xenium_dapi[
    min_x:max_x, min_y:max_y
]  # [second_min_x:second_max_x, second_min_y:second_max_y]
ax1.imshow(img_cropped, vmax=np.percentile(img_cropped, 99.9), cmap="Greys_r")

# Create an empty GeoDataFrame to store adjusted polygons
adjusted_shapes = []

# Iterate through the shapes DataFrame and adjust each polygon
for original_polygon in shapes:
    scaled_polygon = sa.translate(original_polygon, -min_y, -min_x)
    adjusted_shapes.append(scaled_polygon)

adjusted_shapes = gpd.GeoSeries(adjusted_shapes)

for geometry, color in zip(adjusted_shapes, colors):
    if geometry.geom_type == "Polygon":
        patch = plt.Polygon(
            list(zip(*geometry.exterior.xy)),
            facecolor=color,
            edgecolor="none",
            alpha=inside_alpha,
            zorder=1,
        )
        ax1.add_patch(patch)
    elif geometry.geom_type == "MultiPolygon":
        for poly in geometry:
            patch = plt.Polygon(
                list(zip(*poly.exterior.xy)),
                facecolor=color,
                edgecolor="none",
                alpha=inside_alpha,
                zorder=1,
            )
            ax1.add_patch(patch)

# Plot polygon edges with edgecolor based on data values
for geometry, color in zip(adjusted_shapes, colors):
    if geometry.geom_type == "Polygon":
        ax1.plot(*geometry.exterior.xy, color=color, alpha=outside_alpha)
    elif geometry.geom_type == "MultiPolygon":
        for poly in geometry:
            ax1.plot(*poly.exterior.xy, color=color, alpha=outside_alpha)

rectangle2 = Rectangle(
    (second_min_y, second_min_x),
    second_max_y - second_min_y,
    second_max_x - second_min_x,
    linewidth=4,
    edgecolor="white",
    facecolor="none",
    zorder=2,
)
ax1.add_patch(rectangle2)
ax1.set_xlim(0, max_y - min_y)
ax1.set_ylim(0, max_x - min_x)
ax1.invert_yaxis()
# ax1.axis('equal')
ax1.axis("off")
plt.show()

## Part 4: Crypt-villus axis

In [ ]:
plt.figure(figsize=(10, 4), dpi=300)
ax1 = plt.gca()

segmentation_face_color = "crypt_villi_axis"
inside_alpha = 0.34
outside_alpha = 0.34
celltypes = []
ids = np.array([i.split("-")[0] for i in finalized_adata.obs.index.values]).astype(int)
id_df = pd.DataFrame(
    zip(ids, finalized_adata.obs[segmentation_face_color].values),
    columns=["id", segmentation_face_color],
)
transcripts_with_obs = transcripts_df.merge(
    id_df, left_on="split_cell", right_on="id", how="left"
)
transcripts_with_obs = transcripts_with_obs.dropna(axis=0)


print("Making Shapes")
gby = transcripts_with_obs[
    (transcripts_with_obs.split_cell != 0) & (transcripts_with_obs.split_cell != -1)
].groupby("split_cell")


shapes = []
for group in tqdm(gby):
    shapes.append(make_alphashape(group[1][["x", "y"]].values, alpha=0.05))
    ctype = group[1][segmentation_face_color].values[0]
    celltypes.append(ctype)
shapes = gpd.GeoSeries(shapes)

# Generate an example array of numbers (replace this with your own data)
data = np.array(celltypes)

# Define the colormap and normalization
cmap = coolwarm
norm = Normalize(vmin=data.min(), vmax=data.max())

# Create a colormap object
mappable = plt.cm.ScalarMappable(cmap=cmap, norm=norm)

# Map the data to colors
colors = mappable.to_rgba(data)


img_cropped = xenium_dapi[
    min_x:max_x, min_y:max_y
]  # [second_min_x:second_max_x, second_min_y:second_max_y]
ax1.imshow(img_cropped, vmax=np.percentile(img_cropped, 99.9), cmap="Greys_r")

# Create an empty GeoDataFrame to store adjusted polygons
adjusted_shapes = []

# Iterate through the shapes DataFrame and adjust each polygon
for original_polygon in shapes:
    scaled_polygon = sa.translate(original_polygon, -min_y, -min_x)
    adjusted_shapes.append(scaled_polygon)

adjusted_shapes = gpd.GeoSeries(adjusted_shapes)

for geometry, color in zip(adjusted_shapes, colors):
    if geometry.geom_type == "Polygon":
        patch = plt.Polygon(
            list(zip(*geometry.exterior.xy)),
            facecolor=color,
            edgecolor="none",
            alpha=inside_alpha,
            zorder=1,
        )
        ax1.add_patch(patch)
    elif geometry.geom_type == "MultiPolygon":
        for poly in geometry:
            patch = plt.Polygon(
                list(zip(*poly.exterior.xy)),
                facecolor=color,
                edgecolor="none",
                alpha=inside_alpha,
                zorder=1,
            )
            ax1.add_patch(patch)

# Plot polygon edges with edgecolor based on data values
for geometry, color in zip(adjusted_shapes, colors):
    if geometry.geom_type == "Polygon":
        ax1.plot(*geometry.exterior.xy, color=color, alpha=outside_alpha)
    elif geometry.geom_type == "MultiPolygon":
        for poly in geometry:
            ax1.plot(*poly.exterior.xy, color=color, alpha=outside_alpha)

rectangle2 = Rectangle(
    (second_min_y, second_min_x),
    second_max_y - second_min_y,
    second_max_x - second_min_x,
    linewidth=4,
    edgecolor="white",
    facecolor="none",
    zorder=2,
)
ax1.add_patch(rectangle2)
ax1.set_xlim(0, max_y - min_y)
ax1.set_ylim(0, max_x - min_x)
ax1.invert_yaxis()
# ax1.axis('equal')
ax1.axis("off")
plt.show()

## Part 5: epithelial axis

In [ ]:
plt.figure(figsize=(10, 4), dpi=300)
ax1 = plt.gca()

segmentation_face_color = "epithelial_distance_clipped"
inside_alpha = 0.34
outside_alpha = 0.34
celltypes = []
ids = np.array([i.split("-")[0] for i in finalized_adata.obs.index.values]).astype(int)
id_df = pd.DataFrame(
    zip(ids, finalized_adata.obs[segmentation_face_color].values),
    columns=["id", segmentation_face_color],
)
transcripts_with_obs = transcripts_df.merge(
    id_df, left_on="split_cell", right_on="id", how="left"
)
transcripts_with_obs = transcripts_with_obs.dropna(axis=0)


print("Making Shapes")
gby = transcripts_with_obs[
    (transcripts_with_obs.split_cell != 0) & (transcripts_with_obs.split_cell != -1)
].groupby("split_cell")


shapes = []
for group in tqdm(gby):
    shapes.append(make_alphashape(group[1][["x", "y"]].values, alpha=0.05))
    ctype = group[1][segmentation_face_color].values[0]
    celltypes.append(ctype)
shapes = gpd.GeoSeries(shapes)

# Generate an example array of numbers (replace this with your own data)
data = np.array(celltypes)

# Define the colormap and normalization
cmap = coolwarm
norm = Normalize(vmin=data.min(), vmax=data.max() * 0.6)

# Create a colormap object
mappable = plt.cm.ScalarMappable(cmap=cmap, norm=norm)

# Map the data to colors
colors = mappable.to_rgba(data)


img_cropped = xenium_dapi[
    min_x:max_x, min_y:max_y
]  # [second_min_x:second_max_x, second_min_y:second_max_y]
ax1.imshow(img_cropped, vmax=np.percentile(img_cropped, 99.9), cmap="Greys_r")

# Create an empty GeoDataFrame to store adjusted polygons
adjusted_shapes = []

# Iterate through the shapes DataFrame and adjust each polygon
for original_polygon in shapes:
    scaled_polygon = sa.translate(original_polygon, -min_y, -min_x)
    adjusted_shapes.append(scaled_polygon)

adjusted_shapes = gpd.GeoSeries(adjusted_shapes)

for geometry, color in zip(adjusted_shapes, colors):
    if geometry.geom_type == "Polygon":
        patch = plt.Polygon(
            list(zip(*geometry.exterior.xy)),
            facecolor=color,
            edgecolor="none",
            alpha=inside_alpha,
            zorder=1,
        )
        ax1.add_patch(patch)
    elif geometry.geom_type == "MultiPolygon":
        for poly in geometry:
            patch = plt.Polygon(
                list(zip(*poly.exterior.xy)),
                facecolor=color,
                edgecolor="none",
                alpha=inside_alpha,
                zorder=1,
            )
            ax1.add_patch(patch)

# Plot polygon edges with edgecolor based on data values
for geometry, color in zip(adjusted_shapes, colors):
    if geometry.geom_type == "Polygon":
        ax1.plot(*geometry.exterior.xy, color=color, alpha=outside_alpha)
    elif geometry.geom_type == "MultiPolygon":
        for poly in geometry:
            ax1.plot(*poly.exterior.xy, color=color, alpha=outside_alpha)

rectangle2 = Rectangle(
    (second_min_y, second_min_x),
    second_max_y - second_min_y,
    second_max_x - second_min_x,
    linewidth=4,
    edgecolor="white",
    facecolor="none",
    zorder=2,
)
ax1.add_patch(rectangle2)
ax1.set_xlim(0, max_y - min_y)
ax1.set_ylim(0, max_x - min_x)
ax1.invert_yaxis()
# ax1.axis('equal')
ax1.axis("off")
plt.show()

## Part 6: All transcripts

In [ ]:
plt.figure(figsize=(5, 10), dpi=300)
ax1 = plt.gca()

segmentation_face_color = "leiden"
inside_alpha = 0.34
outside_alpha = 0.8
celltypes = []
ids = np.array([i.split("-")[0] for i in finalized_adata.obs.index.values]).astype(int)
id_df = pd.DataFrame(
    zip(ids, finalized_adata.obs[segmentation_face_color].values),
    columns=["id", segmentation_face_color],
)
transcripts_with_obs = transcripts_df_second.merge(
    id_df, left_on="split_cell", right_on="id", how="left"
)
transcripts_with_obs = transcripts_with_obs.dropna(axis=0)


print("Making Shapes")
gby = transcripts_with_obs[
    (transcripts_with_obs.split_cell != 0) & (transcripts_with_obs.split_cell != -1)
].groupby("split_cell")


shapes = []
for group in tqdm(gby):
    shapes.append(make_alphashape(group[1][["x", "y"]].values, alpha=0.05))
    ctype = group[1][segmentation_face_color].values[0]
    cell_location = np.where(
        finalized_adata.obs[segmentation_face_color].cat.categories == ctype
    )[0]
    try:
        celltypes.append(
            finalized_adata.uns[f"{segmentation_face_color}_colors"][cell_location][0]
        )
    except:
        celltypes.append(
            finalized_adata.uns[f"{segmentation_face_color}_colors"][cell_location[0]]
        )
shapes = gpd.GeoSeries(shapes)
colors = ["#D3D3D3" for s in range(len(shapes))]


img_cropped = xenium_dapi[min_x:max_x, min_y:max_y][
    second_min_x:second_max_x, second_min_y:second_max_y
]
ax1.imshow(
    img_cropped,
    vmax=np.percentile(img_cropped, 99.9),
    vmin=np.percentile(img_cropped, 30),
    cmap="Greys_r",
)

# Create an empty GeoDataFrame to store adjusted polygons
adjusted_shapes = []

# Iterate through the shapes DataFrame and adjust each polygon
for original_polygon in shapes:
    scaled_polygon = sa.translate(
        original_polygon, -min_y - second_min_y, -min_x - second_min_x
    )
    adjusted_shapes.append(scaled_polygon)

adjusted_shapes = gpd.GeoSeries(adjusted_shapes)

for geometry, color in zip(adjusted_shapes, colors):
    if geometry.geom_type == "Polygon":
        patch = plt.Polygon(
            list(zip(*geometry.exterior.xy)),
            facecolor=color,
            edgecolor="none",
            alpha=inside_alpha,
            zorder=1,
        )
        ax1.add_patch(patch)
    elif geometry.geom_type == "MultiPolygon":
        for poly in geometry:
            patch = plt.Polygon(
                list(zip(*poly.exterior.xy)),
                facecolor=color,
                edgecolor="none",
                alpha=inside_alpha,
                zorder=1,
            )
            ax1.add_patch(patch)

# Plot polygon edges with edgecolor based on data values
for geometry, color in zip(adjusted_shapes, colors):
    if geometry.geom_type == "Polygon":
        ax1.plot(*geometry.exterior.xy, color=color, linewidth=4, alpha=outside_alpha)
    elif geometry.geom_type == "MultiPolygon":
        for poly in geometry:
            ax1.plot(*poly.exterior.xy, color=color, linewidth=4, alpha=outside_alpha)

transcripts_genes_only = transcripts_df_second

import random


# Function to generate a random color in RGB format
def random_color():
    return "#{:02x}{:02x}{:02x}".format(
        random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)
    )


transcript_colors = [random_color() for _ in range(500)]
pt_size = 1.2
gene_subset = finalized_adata.var.index.values

transcripts_cellpose = pd.read_csv(
    "../data/transcripts/transcripts_figure_5c.csv", index_col=0
)

transcripts_cellpose["x_location_pixels"] = transcripts_cellpose.x.values * (
    1 / pixel_size
)
transcripts_cellpose["y_location_pixels"] = transcripts_cellpose.y.values * (
    1 / pixel_size
)
transcripts_cellpose["feature_name"] = transcripts_cellpose.gene.values

subsetted_indices_second = np.where(
    (transcripts_cellpose.x_location_pixels.values > side1)
    & (transcripts_cellpose.x_location_pixels.values < side2)
    & (transcripts_cellpose.y_location_pixels.values > side3)
    & (transcripts_cellpose.y_location_pixels.values < side4)
)[0]

transcripts_df_second2 = pd.DataFrame(
    zip(
        transcripts_cellpose.x_location_pixels.values[subsetted_indices_second],
        transcripts_cellpose.y_location_pixels.values[subsetted_indices_second],
        transcripts_cellpose.feature_name.values[subsetted_indices_second],
    ),
    columns=["x", "y", "gene"],
)

col_ct = 0
for i in gene_subset:
    transcripts_genes_only_current = transcripts_df_second2[
        transcripts_df_second2["gene"] == i
    ]
    for x, y in zip(
        transcripts_genes_only_current.x.values, transcripts_genes_only_current.y.values
    ):
        circle = patches.Circle(
            (x - (min_y + second_min_y), y - (min_x + second_min_x)),
            radius=pt_size,
            edgecolor="black",
            linewidth=0.01,
            facecolor=transcript_colors[col_ct],
            alpha=1,
            zorder=2,
        )
        ax1.add_patch(circle)
    col_ct += 1

col_ct = 0
for i in gene_subset:
    plt.scatter([], [], c=transcript_colors[col_ct], label=i)
    col_ct += 1

# ax1.set_xlim(0, max_y - min_y)
# ax1.set_ylim(0, max_x - min_x)
# ax1.axis('equal')
ax1.axis("off")
plt.show()

## Part 7: IF and CD8 transcripts

In [ ]:
plt.figure(figsize=(5, 10), dpi=300)
ax1 = plt.gca()

if_channels = [1, 0]
# Rest of the axes

mapped_ims = []
mapped_ims.append(
    np.zeros(
        np.shape(
            IF_image[min_x:max_x, min_y:max_y, if_channels[0]][
                second_min_x:second_max_x, second_min_y:second_max_y
            ]
        )
    )
)
for g in range(len(if_channels)):
    image = IF_image[min_x:max_x, min_y:max_y, if_channels[g]][
        second_min_x:second_max_x, second_min_y:second_max_y
    ]
    min_val = np.min(image)
    max_val = np.max(image)

    normalized_image = (image - min_val) / (max_val - min_val)

    # plt.hist(image)
    # plt.show()

    if if_channels[g] == 2:
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (30, 30))
        # Top Hat Transform
        topHat = cv2.morphologyEx(normalized_image, cv2.MORPH_TOPHAT, kernel)
        # Black Hat Transform
        blackHat = cv2.morphologyEx(normalized_image, cv2.MORPH_BLACKHAT, kernel)

        normalized_image = normalized_image + topHat - blackHat

        normalized_image = normalized_image * 2

    mapped_ims.append(normalized_image)

full_im = np.dstack(mapped_ims)

img_cropped = full_im  # xenium_dapi[min_x:max_x, min_y:max_y][second_min_x:second_max_x, second_min_y:second_max_y]
ax1.imshow(
    img_cropped,
    vmax=np.percentile(img_cropped, 99.9),
    vmin=np.percentile(img_cropped, 30),
    cmap="Greys_r",
)

# adjusted_shapes = gpd.GeoSeries(adjusted_shapes)

for geometry, color in zip(adjusted_shapes, colors):
    if geometry.geom_type == "Polygon":
        patch = plt.Polygon(
            list(zip(*geometry.exterior.xy)),
            facecolor=color,
            edgecolor="none",
            alpha=inside_alpha,
            zorder=1,
        )
        ax1.add_patch(patch)
    elif geometry.geom_type == "MultiPolygon":
        for poly in geometry:
            patch = plt.Polygon(
                list(zip(*poly.exterior.xy)),
                facecolor=color,
                edgecolor="none",
                alpha=inside_alpha,
                zorder=1,
            )
            ax1.add_patch(patch)

# Plot polygon edges with edgecolor based on data values
for geometry, color in zip(adjusted_shapes, colors):
    if geometry.geom_type == "Polygon":
        ax1.plot(*geometry.exterior.xy, color=color, alpha=outside_alpha)
    elif geometry.geom_type == "MultiPolygon":
        for poly in geometry:
            ax1.plot(*poly.exterior.xy, color=color, alpha=outside_alpha)

transcripts_genes_only = transcripts_df_second

import random


# Function to generate a random color in RGB format
def random_color():
    return "#{:02x}{:02x}{:02x}".format(
        random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)
    )


xist = "#00FFFF"
cd8a = "#FEFDFD"  # White
cd8b1 = "#FF00FF"  # Bright Red
itgae = "#32CD32"
cd3d = "#FFD700"
cd3e = "#FF0000"
cd3g = "#997950"
# gzmb = "#FF00FF"

transcript_colors = [cd8a, cd8b1, xist, itgae, cd3d, cd3e, cd3g]

pt_size = 2
gene_subset = ["CD8A", "CD8B", "GZMA", "ITGAE", "CD3D", "CD3E", "CD3G"]
col_ct = 0
for i in gene_subset:
    transcripts_genes_only_current = transcripts_genes_only[
        transcripts_genes_only["gene"] == i
    ]
    for x, y in zip(
        transcripts_genes_only_current.x.values, transcripts_genes_only_current.y.values
    ):
        circle = patches.Circle(
            (x - (min_y + second_min_y), y - (min_x + second_min_x)),
            radius=pt_size,
            edgecolor="black",
            linewidth=1,
            facecolor=transcript_colors[col_ct],
            alpha=1,
            zorder=2,
        )
        ax1.add_patch(circle)
    col_ct += 1

col_ct = 0
for i in gene_subset:
    plt.scatter([], [], c=transcript_colors[col_ct], label=i)
    col_ct += 1

ax1.axis("off")

ax1.legend()
plt.show()